In [8]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

#1. Examine both Open Meteo and Tomorrow and decide which one you'd prefer to use. What drove your decision?
#I decided on Open Meteo because it had better sample code with the documentation.
#The samples give me a starting point to start customizing the code for what I want to do.

#2. What is the URL for the documentation?
#https://open-meteo.com/en/docs

#3. Make a request for the current weather where you are born, or somewhere you've lived.
#West Bloomfield, MI 48324

#documentation shows they want the data in lat/long, not street address
#per Google: 42.5575° N, 83.3614° W

#Here is the code I put together with the help of ChatGPT
import requests
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = (
    f"https://api.open-meteo.com/v1/forecast"
    f"?latitude=42.5575&longitude=-83.3614"
    f"&current_weather=true"
)

# Send the GET request
response = requests.get(url)
data = response.json()

# Parse and print current weather
if 'current_weather' in data:
    weather = data['current_weather']
    print(f"Temperature: {weather['temperature']}°C")
    print(f"Wind Speed: {weather['windspeed']} km/h")
    print(f"Weather Code: {weather['weathercode']}")
    print(f"Time: {weather['time']}")
else:
    print("No current weather data available.")


Temperature: 16.3°C
Wind Speed: 18.2 km/h
Weather Code: 0
Time: 2025-06-10T02:00


In [34]:
#This is the code just cutting and pasting from Open Meteo. 
#The ChatGPT format is much easier to read but I could also fiddle with the Open Meteo to get something prettier-looking
import requests
import requests
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# API request setup
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 42.5575,
    "longitude": -83.3614,
    "current": ["temperature_2m", "apparent_temperature"],
    "temperature_unit": "fahrenheit"
}

# Make request and get response
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

# Print general metadata
print(f"Coordinates: {response.Latitude()}°N, {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()} ({response.TimezoneAbbreviation()})")
print(f"UTC Offset: {response.UtcOffsetSeconds()} seconds")

# Get current weather data
current = response.Current()
current_temperature_2m = current.Variables(0).Value()
current_apparent_temperature = current.Variables(1).Value()
temp_diff = abs(current_temperature_2m - current_apparent_temperature)

# Print weather details
print(f"Current time: {current.Time()}")
print(f"Current temperature: {current_temperature_2m} °F")
print(f"Apparent temperature: {current_apparent_temperature} °F")
print(f"Temperature difference: {temp_diff:.2f} °F")

#5. YES I KNOW THIS IS OUT OF ORDER!
#It made more sense to put this code here rather than below

if current_temperature_2m > current_apparent_temperature:
  print(f"It feels {temp_diff:.2f} degrees colder.")
else:
  print(f"It feels {temp_diff:.2f} degrees warmer.")

#4. Print out the country this is located in.
#Open Meteo doesn't have a country tag, so ChatGPT recommended reverse geocoding using Nominatim:

import requests

latitude = 42.5575
longitude = -83.3614

# Nominatim reverse geocoding URL
url = f"https://nominatim.openstreetmap.org/reverse?lat=42.5575&lon=-83.3614&format=json"

headers = {
    'User-Agent': 'YourAppNameHere (carrierobin@gmail.com)'  # Required by Nominatim usage policy
}

response = requests.get(url, headers=headers)
data = response.json()

# Extract country name
if 'address' in data and 'country' in data['address']:
    country = data['address']['country']
    print(f"Country: {country}")
else:
    print("Country not found.")

Coordinates: 42.548404693603516°N, -83.351806640625°E
Elevation: 286.0 m asl
Timezone: None (None)
UTC Offset: 0 seconds
Current time: 1749525300
Current temperature: 58.871299743652344 °F
Apparent temperature: 54.86722946166992 °F
Temperature difference: 4.00 °F
It feels 4.00 degrees colder.
Country: United States


In [56]:
#6. What is the current temperature at Heathrow?
import requests
import openmeteo_requests
import requests_cache
from retry_requests import retry
import datetime

# Set up cache and retry
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define API request
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 51.4700,
    "longitude": -0.4543,
    "current": ["temperature_2m", "apparent_temperature"],
    "daily": ["temperature_2m_max"],
    "temperature_unit": "fahrenheit",
    "timezone": "Europe/London"
}

# Call the API
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

# General metadata
print(f"Coordinates: {response.Latitude()}°N, {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()} ({response.TimezoneAbbreviation()})")
print(f"UTC Offset: {response.UtcOffsetSeconds()} seconds")

# Extract and print forecast dates
daily = response.Daily()
dates = daily.Time()

# Debugging
print(f"Type of dates: {type(dates)}")
print(f"Value of dates: {dates}")


#7. What URL would I use to get a 3-day forecast at Heathrow?
#https://open-meteo.com/en/docs?hourly=&temperature_unit=fahrenheit&forecast_days=3&longitude=-0.4565&latitude=51.4712
#In the API documentation, it's:
# url = "https://api.open-meteo.com/v1/forecast"
# params = {
# 	"latitude": 51.4712,
# 	"longitude": -0.4565,
# 	"forecast_days": 3,
# 	"temperature_unit": "fahrenheit"
# }
# responses = openmeteo.weather_api(url, params=params)

#8. Print the date of each of the 3 days you're getting a forecast for.
#digging into the API documentation, it's found under "daily"
#says {&daily=} will accept the following values (then lists them). 
#list includes start_date and end_date


Coordinates: 51.47999954223633°N, -0.46000003814697266°E
Elevation: 26.0 m asl
Timezone: b'Europe/London' (b'GMT+1')
UTC Offset: 3600 seconds
Type of dates: <class 'int'>
Value of dates: 1749510000


In [54]:
#Getting all sorts of error messages with the date, ran this to determine what form the dates are in

import requests

url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 51.4700,
    "longitude": -0.4543,
    "daily": ["temperature_2m_max"],
    "timezone": "Europe/London"
}

response = requests.get(url, params=params)
data = response.json()

print(data['daily']['time'])

['2025-06-10', '2025-06-11', '2025-06-12', '2025-06-13', '2025-06-14', '2025-06-15', '2025-06-16']


In [57]:
#updated the entire request with help from ChatGPT
#JFC This took like 8 rounds to the mat to figure this damn thing out.
#I'm not sure whether I need a cigarette or a bourbon. Or both. And I don't smoke!
#8. FINALLY
import requests
from datetime import datetime, date, timedelta

# Open-Meteo API endpoint and parameters for Heathrow Airport
url = "https://api.open-meteo.com/v1/forecast"

# Get today's date in YYYY-MM-DD format
today = date.today()
# Calculate the end date (3 days from today)
end_date = today + timedelta(days=2)  # 3 days total including today

params = {
    "latitude": 51.4700,
    "longitude": -0.4543,
    "daily": ["temperature_2m_max"],
    "timezone": "Europe/London",
    "start_date": today.strftime("%Y-%m-%d"),
    "end_date": end_date.strftime("%Y-%m-%d")
}

# Make the API request
response = requests.get(url, params=params)
data = response.json()

# Extract list of date strings from the daily forecast
dates = data['daily']['time']  # list of date strings

print("3-Day Forecast Dates for Heathrow Airport:")

# Convert and print each date string as datetime.date
for date_str in dates:
    date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()
    print(date_obj)

3-Day Forecast Dates for Heathrow Airport:
2025-06-09
2025-06-10
2025-06-11


In [59]:
#9. Print out the maximum temperature for each of the three days
import requests
from datetime import datetime, date, timedelta

# Open-Meteo API endpoint and parameters for Heathrow Airport
url = "https://api.open-meteo.com/v1/forecast"

# Define 3-day range: today through 2 days from now
today = date.today()
end_date = today + timedelta(days=2)  # inclusive of today

params = {
    "latitude": 51.4700,
    "longitude": -0.4543,
    "daily": ["temperature_2m_max"],
    "timezone": "Europe/London",
    "start_date": today.strftime("%Y-%m-%d"),
    "end_date": end_date.strftime("%Y-%m-%d"),
    "temperature_unit": "fahrenheit"  # or "celsius"
}

# Make the API request
response = requests.get(url, params=params)
data = response.json()

# Extract dates and max temperatures
dates = data['daily']['time']                      # list of date strings
temps = data['daily']['temperature_2m_max']        # list of max temps (floats)

print("3-Day Forecast for Heathrow Airport:\n")

# Loop through both lists and print each date + temperature
for date_str, temp in zip(dates, temps):
    date_obj = datetime.strptime(date_str, '%Y-%m-%d').date()
    print(f"{date_obj} — Max Temp: {temp}°F")
#initialize
highest_temp = -9999
hottest_day = None

for date, temp in zip(dates, temps):
    if temp > highest_temp:
        highest_temp = temp
        hottest_day = date
print(f"The hottest day is {hottest_day} with a temperature of {highest_temp}°F.")

3-Day Forecast for Heathrow Airport:

2025-06-09 — Max Temp: 66.9°F
2025-06-10 — Max Temp: 72.1°F
2025-06-11 — Max Temp: 75.3°F
The hottest day is 2025-06-11 with a temperature of 75.3°F.


In [ ]:
#11. Honestly the Weather API was far easier- the variables were much simpler and didn't require as much converting back and forth.
#I'm guessing you customized the questions to the info available at Weather API
#At Open Meteo, it was more of a free for all
#I would have gotten NOWHERE without relying on ChatGPT. I did the weather API with minimal assistance (finding errors, etc)